In [23]:
import pandas as pd

In [24]:
train=pd.read_csv('/content/drive/MyDrive/ulianov/train_exif.csv')
test=pd.read_csv('/content/drive/MyDrive/ulianov/test_exif.csv')

In [25]:
train['width']=train['x_max']-train['x_min']
train['height']=train['y_max']-train['y_min']

In [26]:
drops=['image_name','class']

train=train.drop(columns=drops)



In [27]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
pd.set_option('display.max_columns', 500)
from catboost import CatBoostRegressor

In [29]:
#print(train['OffsetTimeDigitized'].value_counts())

In [30]:
import numpy as np

def prepare(x):
    x = x.split(' ')
    date = x[0].split(':')
    time = x[1].split(':')
    year,month,day = map(int, date)
    timestamp = pd.Timestamp(f'{day}-{month}-{year}')
    hour = int(time[0])
    dayofyear = timestamp.dayofyear
    dayofweek = timestamp.dayofweek
    if dayofweek >= 5:
        days_to_weekend = 0
    else:
        days_to_weekend = min(dayofweek + 1, abs(5 - dayofweek))
    return year, month, day, hour, dayofyear, dayofweek, dayofweek>=5, (month-1) // 3, days_to_weekend

add_columns = ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', \
               'season', 'days_to_weekend']
train[add_columns] = list(train['DateTime'].apply(prepare))
for col in add_columns:
    train[col] = train[col].astype(np.int32)
  
test[add_columns] = list(test['DateTime'].apply(prepare))
for col in add_columns:
    test[col] = test[col].astype(np.int32)
  
date_drops=['DateTimeOriginal','DateTimeDigitized','DateTime']
train=train.drop(columns=date_drops)
test=test.drop(columns=date_drops)

In [31]:
#print(train)
drops=['Make','Model','DateTime','HostComputer','ExifVersion','DateTimeOriginal',
       'DateTimeDigitized','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','SceneType',
       'LensMake','5']

train=pd.get_dummies(train,columns=['LensModel','1','3','12','16','23','Software'])
test=pd.get_dummies(test,columns=['LensModel','1','3','12','16','23','Software'])

train=pd.get_dummies(train,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])
test=pd.get_dummies(test,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])


#train=train.drop(columns=drops)
#test=test.drop(columns=drops)
print(train)

     distance       x_min        y_min        x_max        y_max      conf  \
0        4.88  219.604050   402.193756   425.297180   455.929901  0.842233   
1        1.54  614.961609  1326.627197  1302.240479  1479.057617  0.864385   
2        3.68  298.759644   461.289703   591.384766   529.101013  0.865617   
3        2.22  501.177734   748.439636   983.731384   856.448853  0.874792   
4        3.73  294.875153   471.319153   591.095154   539.254272  0.868262   
..        ...         ...          ...          ...          ...       ...   
525      3.00  306.027252   650.916504   604.249329   720.184814  0.865165   
526      3.46  269.867737   575.216919   522.212463   635.551270  0.865655   
527      3.91  249.696381   501.747467   478.498352   556.968689  0.857690   
528      5.20  198.687332   382.187531   368.827576   425.248566  0.837795   
529      5.54  181.400436   356.746887   344.161957   398.403748  0.829634   

     ResolutionUnit  ExifOffset  Orientation  XResolution  YRes

In [32]:
X=train.drop('distance',axis=1)
y=train['distance']
seed=42
cb=CatBoostRegressor(iterations=20000,random_state=seed,task_type='GPU',learning_rate=0.001)
cb.fit(X,y,verbose=1000)


test_vals=test.drop('image_name',axis=1)
preds=cb.predict(test_vals)

sub_d={
    'image_name':list(test[['image_name']].values),
    'distance':preds
}

sub=pd.DataFrame(sub_d)
sub['image_name']=list(map(lambda x:x[0],sub['image_name']))
print(sub)
sub.to_csv('/content/drive/MyDrive/ulianov/res.csv',index=False,sep=';')

0:	learn: 1.7558219	total: 34.4ms	remaining: 11m 27s
1000:	learn: 0.8904983	total: 19.6s	remaining: 6m 12s
2000:	learn: 0.5628100	total: 29.4s	remaining: 4m 24s
3000:	learn: 0.4454547	total: 39.1s	remaining: 3m 41s
4000:	learn: 0.3951796	total: 48.8s	remaining: 3m 14s
5000:	learn: 0.3673935	total: 58.5s	remaining: 2m 55s
6000:	learn: 0.3478656	total: 1m 8s	remaining: 2m 38s
7000:	learn: 0.3314194	total: 1m 17s	remaining: 2m 24s
8000:	learn: 0.3167683	total: 1m 27s	remaining: 2m 11s
9000:	learn: 0.3044409	total: 1m 38s	remaining: 2m
10000:	learn: 0.2939422	total: 1m 48s	remaining: 1m 48s
11000:	learn: 0.2838895	total: 1m 57s	remaining: 1m 36s
12000:	learn: 0.2751698	total: 2m 10s	remaining: 1m 26s
13000:	learn: 0.2671510	total: 2m 20s	remaining: 1m 15s
14000:	learn: 0.2602478	total: 2m 29s	remaining: 1m 4s
15000:	learn: 0.2540602	total: 2m 39s	remaining: 53.2s
16000:	learn: 0.2484100	total: 2m 49s	remaining: 42.4s
17000:	learn: 0.2431100	total: 2m 59s	remaining: 31.7s
18000:	learn: 0.23